In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Multi-Class Sentiment Analysis
This notebook follows this tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis.

In [24]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

# Set `batch_first=True` in the `Field`.
TEXT = Field(tokenize='spacy', include_lengths=True, batch_first=True)
LABEL = LabelField()

# `filter_pred`: Use only examples for which `filter_pred(example)` is True.
# Note: Some sentences have lengths shorter than CNN kernel size...
train_data, test_data = torchtext.datasets.TREC.splits(TEXT, LABEL, fine_grained=False, root='data', 
                                                       filter_pred=lambda ex: len(ex.text)>=4)
train_data, valid_data = train_data.split()

In [25]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'ENTY': 0, 'HUM': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5})


In [26]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [27]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[   4,    5,    3,  267,    6,    3,   91,   27,    9,  449, 1929,   49,
            2,    1,    1,    1,    1,    1,    1,    1],
        [  10,   24,  225,   21,  561, 1198,    6,    3, 3827,   42,    2,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  10,   31,   28,   60,  306,   11,  732,  248,    2,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  47, 1351,   12, 3490, 4770,  974,    6, 5934,    2,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [  26,   20,    3,   43,   99,   33,    2,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   4,    5,    8,  106,   17, 4981,   14,  930,   14,   93,  591,   34,
         7353,    3, 4787, 7057,    6, 6907, 6288,    2],
        [  52,    3, 1406,    6,    9,    9,   32,  681,   19,   44,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   4,  209, 1168, 

# Build Model

In [28]:
class Classifier1D(nn.Module):
    def __init__(self, in_dim, emb_dim, n_filters, filter_sizes, out_dim, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        # (2) Regard different embedding dimensions as different channels. 
        # Thus, it reduces to an 1D convolution. 
        self.convs = nn.ModuleList([
            nn.Conv1d(emb_dim, n_filters, kernel_size=fs)
            for fs in filter_sizes])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_filters*len(filter_sizes), out_dim)

    def forward(self, text, text_lens):
        # `text_lens` would be useful when mean-pooling applied. 
        # text/mask: (batch, step)
        mask = text.eq(self.emb.padding_idx)
        # embedded: (batch, step, emb)
        embedded = self.emb(text)
        
        pooled = []
        for conv_i in self.convs:
            # Note: The kernel size may exceed the sequence length, which leads to invalid output. 
            # conved_i: (batch, channel=n_filters, step-(kernel_size-1))
            conved_i = F.relu(conv_i(embedded.permute(0, 2, 1)))
            # mask_i: (batch, 1, step-(kernel_size-1))
            mask_i = mask[:, (conv_i.kernel_size[0]-1):].unsqueeze(1)
            # pooled_i: (batch, n_filters)
            pooled_i, _ = conved_i.masked_fill(mask_i, -np.inf).max(dim=-1)
            pooled.append(pooled_i)

        concated = self.dropout(torch.cat(pooled, dim=-1))
        return self.fc(concated)

In [29]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2, 3, 4]
OUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier1d = Classifier1D(IN_DIM, EMB_DIM, N_FILTERS, FILTER_SIZES, OUT_DIM, 
                           DROPOUT, PAD_IDX).to(device)
print(count_parameters(classifier1d))

classifier = classifier1d

838206


In [30]:
# Initialize Embeddings with Pre-Trained Vectors
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1638,  0.6046,  1.0789, -1.2731, -0.7759,  0.3997,  0.3817, -0.1759],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-1.3771, -0.9446,  1.1019,  1.7499, -1.1453,  0.6823,  0.8769,  0.8717]],
       grad_fn=<SliceBackward>)


# Train Model

In [31]:
optimizer = optim.Adam(classifier.parameters())
loss_func = nn.CrossEntropyLoss().to(device)

In [32]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (preds.argmax(-1) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (preds.argmax(-1) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [33]:
import time
N_EPOCHS = 5
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), 'models/tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.260 | Train Acc: 50.82%
	 Val. Loss: 0.940 |  Val. Acc: 67.71%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.835 | Train Acc: 70.04%
	 Val. Loss: 0.748 |  Val. Acc: 73.46%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.642 | Train Acc: 78.02%
	 Val. Loss: 0.631 |  Val. Acc: 77.41%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.480 | Train Acc: 84.09%
	 Val. Loss: 0.554 |  Val. Acc: 80.47%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.372 | Train Acc: 88.48%
	 Val. Loss: 0.511 |  Val. Acc: 81.54%


# Check Embeddings
* The Embeddings of `unk` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [35]:
print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.1970,  0.5554,  1.1111, -1.2733, -0.8065,  0.3944,  0.4006, -0.1740],
        [-0.0356, -0.3056,  0.7477, -0.4321,  0.0396,  0.0651, -0.4076,  0.2948],
        [-1.3738, -0.9117,  1.0721,  1.7182, -1.1496,  0.6596,  0.8557,  0.8612]],
       grad_fn=<SliceBackward>)
